# Imports

In [1]:
import numpy as np

from data_worker.data_worker import unpickle, unpack_data, combine_batches, split_into_batches
from torch_lib.data_worker import suit4torch
from torch_lib.Interface import Interface
from torch_lib.Nets import LargeNet, MediumNet, SmallNet

# Data

In [2]:
batches_names = [
    'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4',
    'data_batch_5'
]

data_batches = [
    unpickle(f'datasets/cifar-10-batches-py/{batch_name}') for batch_name
    in batches_names]

unpacked_batches = [
    (unpack_data(data_batch)) for data_batch
    in data_batches]

print(unpacked_batches[0][0].shape)

X, Y = combine_batches(unpacked_batches)

X_torch, Y_torch = suit4torch(X, Y)

batches = split_into_batches(X_torch, Y_torch, 3)

# torch_batches = [(suit4torch(X, Y)) for X, Y in batches]

X_batch0 = batches[0][0]
Y_batch0 = batches[0][1]

print(X.shape, Y.shape)

(10000, 32, 32, 3)
(32, 32, 3)
(50000, 32, 32, 3) (50000,)


# Train network

In [3]:
# net = SmallNet()
# net = MediumNet()
net = LargeNet()

In [4]:
net_interface = Interface(net)

In [ ]:
net_interface.train_net(batches, 1, verbose=False)

# Save weights

In [5]:
# PATH = 'saved_nets/saved_torch/small_v1.pth'
# PATH = 'saved_nets/saved_torch/medium_v1.pth'
PATH = 'saved_nets/saved_torch/large_v1.pth'

In [ ]:
net_interface.save_weights(PATH)

# Predict

In [ ]:
preds = net_interface.predict_net(X_batch0)
preds = preds.detach().numpy()

In [ ]:
print(preds)
print(np.argmax(preds, axis=1), Y_batch0)

# Evaluate Accuracy

In [9]:
acc, N = net_interface.eval_acc_net(X_torch, Y_torch)
print(acc, N)

0.1 50000


# Load weights

In [6]:
net_interface.load_weights(PATH)

In [7]:
preds = net_interface.predict_net(X_torch)
preds = preds.detach().numpy()

c:\users\tronc\appdata\local\programs\python\python39\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [8]:
print(preds)
print(np.argmax(preds, axis=1), Y_torch)

[[-0.03193782 -0.02609414 -0.09416538 ... -0.04211621  0.0363774
  -0.00137505]
 [-0.03180882 -0.02602005 -0.09423827 ... -0.04228754  0.0366769
  -0.00128849]
 [-0.03197443 -0.02589439 -0.09421802 ... -0.04241734  0.03658268
  -0.0012246 ]
 ...
 [-0.03207375 -0.02579184 -0.0941498  ... -0.04242862  0.03647955
  -0.00118262]
 [-0.0313148  -0.02602111 -0.09421885 ... -0.04214914  0.03719144
  -0.00123877]
 [-0.03213761 -0.0258862  -0.09394593 ... -0.04200218  0.03606745
  -0.00131202]]
[8 8 8 ... 8 8 8] tensor([6, 9, 9,  ..., 9, 1, 1])


# Run forward function

In [21]:
from threading import Thread
import time
import psutil
import subprocess


class GPUreader(Thread):
    
    def __init__(self):
        self.process = subprocess.Popen(['while (1) { ps | sort -desc cpu | select -first 30; sleep -seconds 2; cls }'], stdout=subprocess.PIPE)
        self.stopped = True
        super().__init__()
    
    def __enter__(self):
        return self
    
    def __exit__(self, exception_type, exception_value, traceback):
        self.stop()
    
    def start(self):
        self.stopped = False
        super().start()
    
    def stop(self):
        self.stopped = True
        self.stop()
    
    def run(self):
        while not self.stopped:
            print(self.process.stdout.readline().strip())


class CPUtracker(Thread):
    
    def __init__(self, Ts):
        self.Ts = Ts
        self.stopped = True
        self.start_time = None
        self.values = []
        super().__init__()
    
    def __enter__(self):
        self.start()
        return self
    
    def __exit__(self, exception_type, exception_value, traceback):
#         print(exception_type, exception_value, traceback)
        self.stop()
    
    def start(self):
        self.stopped = False
        self.start_time = time.time()
        super().start()
    
    def stop(self):
        self.stopped = True

    def run(self):
        while not self.stopped:
            mem = psutil.virtual_memory()
            cpu_percent = psutil.cpu_percent()
            print(mem)
            print(f'{time.time():.8f}')
            self.values.append((time.time() - self.start_time, mem.percent, cpu_percent))
            time.sleep(self.Ts)
    
    def get_values(self):
        return self.values
        
    

In [22]:
with GPUreader() as g:
    pass

FileNotFoundError: [WinError 2] El sistema no puede encontrar el archivo especificado

In [19]:
import sys
import os
import torch
import time


def execute_net(net_interface, X_data, Y_data, batch_size, priority, loops=1, device='cuda', metrics=['ex_time', 'cpu_max', 'gpu_max', 'mem_max'], echo=True):
    if sys.platform == 'linux':
        os.nice(priority)

    device = torch.device('cpu')
    if device == 'cuda':
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        if echo and not torch.cuda.is_available():
            print("'cuda' device is not available. Using 'cpu' instead.")

    batches = split_into_batches(X_data, Y_data, batch_size)
    
    batch_times = []
    
#     cpu_tracker = CPUtracker(0.1)
#     cpu_tracker.start()
    with CPUtracker(0.1) as cpu_tracker:
    
        for loop in range(loops):
            if echo:
                print('loop:', loop)
            for X_batch, Y_batch in batches:
                start_time = time.time()
                Y_pred = net_interface.predict_net(X_batch)
                batch_time = time.time() - start_time
                if echo:
                    print(f'batch_time: {batch_time:.8f}')
                batch_times.append(batch_time)
        
        print(cpu_tracker.get_values())

#     cpu_tracker.stop()
    
#     print(cpu_tracker.get_values())

execute_net(net_interface, X_torch, Y_torch, 10, 0)
    
    

svmem(total=8390193152, available=2978295808, percent=64.5, used=5411897344, free=2978295808)
1634636407.66098642
loop: 0
batch_time: 0.00299358
batch_time: 0.00299048
batch_time: 0.00199413
batch_time: 0.00132084
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.01565266
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.01559281
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.01561952
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batc

batch_time: 0.00199485
batch_time: 0.00199509
batch_time: 0.00299048
batch_time: 0.00199413
batch_time: 0.00199556
batch_time: 0.00199533
svmem(total=8390193152, available=2971930624, percent=64.6, used=5418262528, free=2971930624)
1634636408.41363382
batch_time: 0.00398898
batch_time: 0.00199604
batch_time: 0.00199223
batch_time: 0.00199461
batch_time: 0.00199556
batch_time: 0.00099754
batch_time: 0.00199556
batch_time: 0.00099564
batch_time: 0.00199270
batch_time: 0.00199437
batch_time: 0.00199509
batch_time: 0.00398946
batch_time: 0.00199866
batch_time: 0.00099468
batch_time: 0.00199509
batch_time: 0.00199294
batch_time: 0.00199699
batch_time: 0.00099683
batch_time: 0.00200057
batch_time: 0.00198817
batch_time: 0.00299048
batch_time: 0.00199509
batch_time: 0.00199962
batch_time: 0.00099254
batch_time: 0.00099921
batch_time: 0.00199580
batch_time: 0.00099683
batch_time: 0.00099707
batch_time: 0.00300193
batch_time: 0.00098753
batch_time: 0.00199509
batch_time: 0.00200629
batch_time: 

batch_time: 0.00299215
batch_time: 0.00299215
batch_time: 0.00199509
batch_time: 0.00299215
batch_time: 0.00299263
batch_time: 0.00200081
batch_time: 0.00299287
batch_time: 0.00400567
batch_time: 0.00198889
batch_time: 0.00199389
batch_time: 0.00299478
svmem(total=8390193152, available=2971639808, percent=64.6, used=5418553344, free=2971639808)
1634636409.18260145
batch_time: 0.00299048
batch_time: 0.00199580
batch_time: 0.00200367
batch_time: 0.00199199
batch_time: 0.00299072
batch_time: 0.00299311
batch_time: 0.00100327
batch_time: 0.00099635
batch_time: 0.00398803
batch_time: 0.00199604
batch_time: 0.00199580
batch_time: 0.00299215
batch_time: 0.00299144
batch_time: 0.00199914
batch_time: 0.00398350
batch_time: 0.00199676
batch_time: 0.00199914
batch_time: 0.00299454
batch_time: 0.00299597
batch_time: 0.00198412
batch_time: 0.00399995
batch_time: 0.00198507
batch_time: 0.00199270
batch_time: 0.00299215
batch_time: 0.00299191
batch_time: 0.00199366
batch_time: 0.00299954
batch_time: 

batch_time: 0.00399113
batch_time: 0.00398660
batch_time: 0.00199366
batch_time: 0.00099778
batch_time: 0.00099778
batch_time: 0.00199676
batch_time: 0.00199366
batch_time: 0.00200486
batch_time: 0.00199509
batch_time: 0.00299239
batch_time: 0.00398874
batch_time: 0.00198364
batch_time: 0.00199389
batch_time: 0.00498438
batch_time: 0.00197506
batch_time: 0.00199461
batch_time: 0.00299382
svmem(total=8390193152, available=2971471872, percent=64.6, used=5418721280, free=2971471872)
1634636409.95112395
batch_time: 0.00298953
batch_time: 0.00400186
batch_time: 0.00397825
batch_time: 0.00199556
batch_time: 0.00199533
batch_time: 0.00199413
batch_time: 0.00199413
batch_time: 0.00398922
batch_time: 0.00299263
batch_time: 0.00199413
batch_time: 0.00299311
batch_time: 0.00199771
batch_time: 0.00199127
batch_time: 0.00499296
batch_time: 0.00199461
batch_time: 0.00200415
batch_time: 0.00199008
batch_time: 0.00200510
batch_time: 0.00198293
batch_time: 0.00397849
batch_time: 0.00199485
batch_time: 

batch_time: 0.00000000
batch_time: 0.00653338
batch_time: 0.00199461
batch_time: 0.00199533
batch_time: 0.00199509
batch_time: 0.00299263
batch_time: 0.00398993
batch_time: 0.00298977
batch_time: 0.00199461
batch_time: 0.00199509
batch_time: 0.00199461
batch_time: 0.00199437
batch_time: 0.00199461
batch_time: 0.00199461
batch_time: 0.00199699
batch_time: 0.00098562
batch_time: 0.00099754
batch_time: 0.00199556
batch_time: 0.00099564
batch_time: 0.00199533
batch_time: 0.00199509
batch_time: 0.00099659
batch_time: 0.00199461
batch_time: 0.00199580
batch_time: 0.00049901
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.01562262
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
svmem(total

...

batch_time: 0.00498629
batch_time: 0.00299311
batch_time: 0.00199437
batch_time: 0.00199437
batch_time: 0.00199461
batch_time: 0.00199485
batch_time: 0.00198436
batch_time: 0.00199509
batch_time: 0.00299406
batch_time: 0.00199413
batch_time: 0.00199747
batch_time: 0.00199556
batch_time: 0.00300026
batch_time: 0.00199842
batch_time: 0.00298882
batch_time: 0.00199389
batch_time: 0.00199699
batch_time: 0.00199461
batch_time: 0.00199842
batch_time: 0.00300026
batch_time: 0.00199580
batch_time: 0.00199461
batch_time: 0.00199413
batch_time: 0.00199389
batch_time: 0.00199485
batch_time: 0.00301313
batch_time: 0.00297332
batch_time: 0.00199270
batch_time: 0.00199485
batch_time: 0.00199556
batch_time: 0.00199389
batch_time: 0.00199509
svmem(total=8390193152, available=2972381184, percent=64.6, used=5417811968, free=2972381184)
1634636414.66988897
batch_time: 0.00299287
batch_time: 0.00299191
batch_time: 0.00199413
batch_time: 0.00199461
batch_time: 0.00199485
batch_time: 0.00199461
batch_time: 

batch_time: 0.00199652
batch_time: 0.00199819
batch_time: 0.00099373
batch_time: 0.00099850
batch_time: 0.00199389
batch_time: 0.00199461
batch_time: 0.00199556
batch_time: 0.00299072
batch_time: 0.00199509
batch_time: 0.00199604
batch_time: 0.00199628
batch_time: 0.00199270
batch_time: 0.00299215
batch_time: 0.00199413
batch_time: 0.00099730
batch_time: 0.00199533
batch_time: 0.00099635
batch_time: 0.00199485
batch_time: 0.00200605
batch_time: 0.00098658
batch_time: 0.00100017
batch_time: 0.00200081
batch_time: 0.00198340
batch_time: 0.00199461
batch_time: 0.00199437
batch_time: 0.00199413
batch_time: 0.00199461
batch_time: 0.00199413
batch_time: 0.00199509
batch_time: 0.00199437
batch_time: 0.00205421
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00932145
batch_time: 0.00199485
batch_time: 0.00199437
batch_time: 0.00199580
batch_time: 0.00199437
batch_time: 0.00199461
batch_time: 0.00199580
svmem(total=8390193152, available=

batch_time: 0.00300002
batch_time: 0.00298452
batch_time: 0.00199461
batch_time: 0.00299048
batch_time: 0.00199556
batch_time: 0.00199699
batch_time: 0.00199389
batch_time: 0.00199533
batch_time: 0.00199652
batch_time: 0.00298858
batch_time: 0.00299335
batch_time: 0.00199294
batch_time: 0.00199461
batch_time: 0.00199533
batch_time: 0.00199461
batch_time: 0.00200033
batch_time: 0.00099134
batch_time: 0.00200343
batch_time: 0.00198603
batch_time: 0.00199485
batch_time: 0.00199413
batch_time: 0.00199509
batch_time: 0.00200009
batch_time: 0.00199008
batch_time: 0.00099635
batch_time: 0.00200057
batch_time: 0.00286961
batch_time: 0.00199318
batch_time: 0.00199604
batch_time: 0.00298834
batch_time: 0.00199604
batch_time: 0.00199389
batch_time: 0.00199485
batch_time: 0.00299287
batch_time: 0.00299668
batch_time: 0.00099826
batch_time: 0.00099778
batch_time: 0.00099730
batch_time: 0.00199437
svmem(total=8390193152, available=2973782016, percent=64.6, used=5416411136, free=2973782016)
163463641

batch_time: 0.00198841
batch_time: 0.00299311
batch_time: 0.00199366
batch_time: 0.00199771
batch_time: 0.00199080
batch_time: 0.00200319
batch_time: 0.00299358
batch_time: 0.00199652
batch_time: 0.00099897
batch_time: 0.00199652
batch_time: 0.00199389
batch_time: 0.00299311
batch_time: 0.00199461
batch_time: 0.00199151
batch_time: 0.00199461
svmem(total=8390193152, available=2972499968, percent=64.6, used=5417693184, free=2972499968)
1634636417.06232166
batch_time: 0.00299215
batch_time: 0.00199485
batch_time: 0.00299144
batch_time: 0.00199580
batch_time: 0.00099802
batch_time: 0.00199437
batch_time: 0.00099754
batch_time: 0.00099874
batch_time: 0.00199413
batch_time: 0.00099778
batch_time: 0.00099778
batch_time: 0.00199437
batch_time: 0.00199437
batch_time: 0.00199485
batch_time: 0.00199461
batch_time: 0.00100017
batch_time: 0.00199270
batch_time: 0.00199437
batch_time: 0.00205326
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 

batch_time: 0.00000000
batch_time: 0.00560737
batch_time: 0.00199389
batch_time: 0.00199461
batch_time: 0.00099683
batch_time: 0.00099659
batch_time: 0.00100732
batch_time: 0.00198674
batch_time: 0.00299358
batch_time: 0.00200105
batch_time: 0.00199175
batch_time: 0.00099587
batch_time: 0.00199556
batch_time: 0.00200129
batch_time: 0.00198603
batch_time: 0.00302219
batch_time: 0.00196600
batch_time: 0.00199723
batch_time: 0.00199461
batch_time: 0.00199556
svmem(total=8390193152, available=2974789632, percent=64.5, used=5415403520, free=2974789632)
1634636417.82997394
batch_time: 0.00299120
batch_time: 0.00299287
batch_time: 0.00299168
batch_time: 0.00199413
batch_time: 0.00199461
batch_time: 0.00199795
batch_time: 0.00199151
batch_time: 0.00099564
batch_time: 0.00199819
batch_time: 0.00199270
batch_time: 0.00099659
batch_time: 0.00199580
batch_time: 0.00199437
batch_time: 0.00099778
batch_time: 0.00199389
batch_time: 0.00199461
batch_time: 0.00199437
batch_time: 0.00199318
batch_time: 